In [1]:
import os
import json
import pickle
import numpy as np
import pandas as pd
from tqdm import tqdm

import torch

In [8]:
with open("./human_embeddings.pkl", 'rb') as file:
    human_emb = pickle.load(file)
with open("./mouse_embeddings.pkl", 'rb') as file:
    mouse_emb = pickle.load(file)

import copy
emb = copy.deepcopy(human_emb)

emb.update(mouse_emb)

TOKEN_DICT_DIR = "../../src/token_dictionary.csv"
token_dict = pd.read_csv(TOKEN_DICT_DIR)

Stack human mouse embeddings

Fill with means if this gene doesn't have an embedding

In [32]:
length = len(list(emb.values())[0]['embedding'])
ids = emb.keys()

gpt_embeddings = []
zero_mark = []
for id in tqdm(token_dict["token_id"][token_dict["ensembl_id"].str.startswith(("ENSG", "ENSMUSG"))]):
    if id in ids:
        if 'embedding' in emb[id].keys():
            gene_emb = emb[id]['embedding']
        else:
            gene_emb = emb[id]['gpt_embedding']
    else:
        gene_emb = [0.]*length
        zero_mark.append(id)
    gpt_embeddings.append(gene_emb)

human_means = np.zeros(length)
for k,v in tqdm(human_emb.items()):
    if 'embedding' in v.keys():
        human_means += v['embedding']
    else:
        human_means += v['gpt_embedding']
human_means /= len(human_emb)
human_means = human_means.tolist()

mouse_means = np.zeros(length)
for k,v in tqdm(mouse_emb.items()):
    if 'embedding' in v.keys():
        mouse_means += v['embedding']
    else:
        mouse_means += v['gpt_embedding']
mouse_means /= len(mouse_emb)
mouse_means = mouse_means.tolist()

human_ids = token_dict["token_id"][token_dict["ensembl_id"].str.startswith("ENSG")]
mouse_ids = token_dict["token_id"][token_dict["ensembl_id"].str.startswith("ENSMUSG")]

for idx in zero_mark:
    if idx in human_ids:
        gpt_embeddings[idx] = human_means
    elif idx in mouse_ids:
        gpt_embeddings[idx] = mouse_means

100%|██████████| 27443/27443 [00:01<00:00, 22523.24it/s]


In [33]:
gpt_embeddings = torch.tensor(gpt_embeddings, dtype=torch.float32)
torch.save(gpt_embeddings, "../../src/gpt_embeddings.bin")